# Data preprocessing - dáta **regzam**

V tomto notebooku si načítame dáta *regzam_portalvs.xml*, upravíme ich do slušnej formy pre ďalšie analýzy a následne uložíme v tejto upravenej forme do excelu/csv.

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

regzam dáta sa načítavajú z **nested xml**, elementy *Employment* a *Guarantee* majú ešte vnorené elementy, čo nám trochu sťažuje načítavanie, viď nižšie:

In [2]:
regzam_data = pd.read_xml('../data/original/regzam_portalvs.xml')
display(regzam_data)

,Id,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment,Guarantee
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,NaN,NaN
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,NaN,NaN
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,NaN,NaN
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,NaN,NaN
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,NaN,NaN
...,...,...,...,...,...,...,...,...
32730,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,NaN,NaN
32731,13500390262,Marcela,Harčárová,None,Mgr.,None,NaN,NaN
32732,17108478442,Jana,Juhásová,None,Mgr.,None,NaN,NaN
32733,15917135755,Miriama,Kubušová,None,Mgr.,None,NaN,NaN


- Skúšali sme pracovať s argumentom *stylesheet* pandasovej funkcie read_xml, no bolo ťažké vytvoriť stylesheet, ktorý by vedel načítať nielen vnorené elementy Empolyment a Guarantee, ale aj ich ďalšie vnorené elementy, ako napr. Workplace/University.
- Preto sme napokon zvolili iný prístup:

- Načítame xml ako string:

In [3]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import tostring

tree = ET.parse('../data/original/regzam_portalvs.xml')
tree = tree.getroot()  # type <class 'xml.etree.ElementTree.Element'>
t = tostring(tree, encoding='utf8', method='xml')

- Teraz treba dáta "sploštiť" (anglicky: "flatten"), aby sme sa zbavili vnorení:

- Prvý pokus bol načítať dáta do slovníka, ktorý mal ako keys názvy stĺpcov a values boli príslušné listy s hodnotami pre daný stĺpec. 
Rekurzívne sme získali "sploštený" slovník. Tento prístup mal však problém, že pre tých zamestnancov, ktorým chýbali niektoré údaje (elementy), nepridalo do príslušného "stĺpca" nič,
sĺpce mali teda rozličné dĺžky a stratili sme informáciu o tom, ktorý údaj prislúcha ktorému zamestnancovi. 
- Druhý pokus už bol úspešnejší, lebo sme dáta načítali do slovníka slovníkov, teda slovník result má ako keys identifikačné čísla zamestnancov, a ako hodnoty slovník s jednotlivými údajmi pre daného zamestnanca:

In [4]:
from collections import defaultdict
from lxml import etree

def xml_to_dict(root):
    """
    Convert the nested xml to a "flat" dictionary.
    
    Parameters:
    -----------
    root : <class 'lxml.etree._Element'> object
        The loaded nested xml.
        
    Returns:
    --------
    result : dictionary
        The flat dictionary.
    
    """
    result = defaultdict(dict)
    for child in root:
        row_id = child.attrib['Id']
        result[row_id] = xml_to_dict_helper(child)
        for key in ['GivenName', 'FamilyName', 'DegreePedagogicalAcademic', 
                    'DegreeMaster', 'DegreeDoctor', 'Employment_Workplace_University', 
                    'Employment_Type', 'Employment_Subtype', 'Employment_BeginDate', 
                    'Employment_Ratio', 'Guarantee_StudyProgram', 'Guarantee_University', 
                    'Guarantee_Faculty', 'Guarantee_FormOfStudy', 'Guarantee_LevelOfStudy', 
                    'Employment_Workplace_Faculty', 'Employment_EndDate']:
            try:
                if len(result[row_id][key]) == 1 and key[:9] != "Guarantee":
                    result[row_id][key] = result[row_id][key][0]
                elif len(result[row_id][key]) == 0:
                    result[row_id][key] = None
            except KeyError:
                result[row_id][key] = None
    return result

def xml_to_dict_helper(root):
    # Create a defaultdict to store the result
    result = defaultdict(list)
    # Iterate over the root element's children
    for child in root:
        # Get the child's tag name and text
        tag = child.tag
        text = child.text

        # If the child has children, recursively process them
        if child.getchildren():
            sub_dict = xml_to_dict_helper(child)
            for k, v in sub_dict.items():
                result[tag + '_' + k].extend(v)
        else:
            # If the child has no children, store its tag and text in the result
            result[tag].append(text)

    return result

In [5]:
# Parse the XML document into an <class 'lxml.etree._Element'> object
root = etree.fromstring(t)

# Convert the ElementTree object to a dictionary
result = xml_to_dict(root)

In [6]:
df = pd.DataFrame.from_dict(result).transpose().reset_index().rename(columns={"index": "Employee_ID"})
display(df)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,None,None,None,None,None,None,None
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,None,None,None,None,None,None,None,None,None,None,None,None
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,None,None,None,None,None,None,None,None,None,None,None,None
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"[bezpečnostnoprávna ochrana osôb a majetku, be...","[Akadémia Policajného zboru, Akadémia Policajn...","[None, None, None, None, None, None, None]","[1, 2, 1, 2, 2, 1, 2]","[1, 1, 2, 2, 2, 3, 3]",None,None
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,None,None,None,None,None,None,None
32726,13500390262,Marcela,Harčárová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None
32727,17108478442,Jana,Juhásová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None
32728,15917135755,Miriama,Kubušová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None


Možno trochu nevýhodne sme načítali dáta o Guarantee, ale až neskôr sa ukáže, či tieto dáta potrebujeme v inej forme a teda či sa nám oplatí investovať energiu do zlepšenia.
Môžeme ale aspoň skontrolovať, či listy, ktoré sme uložili do Guarantee stĺpcov, majú pri všetkých riadkoch rovnakú dĺžku:

In [7]:
for index, row in df.iterrows():
    if row["Guarantee_StudyProgram"] is not None:
        if len(row["Guarantee_StudyProgram"]) == len(row["Guarantee_University"]) == len(row["Guarantee_Faculty"]) == len(row["Guarantee_FormOfStudy"]) == len(row["Guarantee_LevelOfStudy"]):
            pass
        else:
            print("The following row has a different number of records in Guarantee columns:", row)

## Pridanie odvodených sĺpcov:

In [8]:
df["Guarantee"] = df.apply(lambda row: row["Guarantee_StudyProgram"] is not None, axis=1)
display(df)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate,Guarantee
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,None,None,None,None,None,None,None,False
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,None,None,None,None,None,None,None,None,None,None,None,None,False
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,None,None,None,None,None,None,None,None,None,None,None,None,False
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"[bezpečnostnoprávna ochrana osôb a majetku, be...","[Akadémia Policajného zboru, Akadémia Policajn...","[None, None, None, None, None, None, None]","[1, 2, 1, 2, 2, 1, 2]","[1, 1, 2, 2, 2, 3, 3]",None,None,True
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,None,None,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,None,None,None,None,None,None,None,False
32726,13500390262,Marcela,Harčárová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None,False
32727,17108478442,Jana,Juhásová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None,False
32728,15917135755,Miriama,Kubušová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None,False


Zmeníme poradie stĺpcov:

In [9]:
df = df[['Employee_ID', 'FamilyName', 'GivenName', 'DegreePedagogicalAcademic', 'DegreeMaster', 'DegreeDoctor', 
         'Employment_Workplace_University', 'Employment_Workplace_Faculty', 
         'Employment_Type', 'Employment_Subtype', 'Employment_BeginDate', 'Employment_EndDate', 'Employment_Ratio', 
         'Guarantee', 'Guarantee_StudyProgram', 'Guarantee_University', 'Guarantee_Faculty', 'Guarantee_FormOfStudy', 'Guarantee_LevelOfStudy'
        ]]

In [10]:
df.to_csv('../data/preprocessed/regzam.csv')

## **Dáta pre analýzu o pracoviskách:**

## Vyberme iba aktuálne dáta:

In [11]:
current_employees = df[df['Employment_BeginDate'].notna()]
current_employees = current_employees[current_employees['Employment_EndDate'].isna()].reset_index()
current_employees = current_employees[['Employee_ID', 'FamilyName', 'GivenName', 'DegreePedagogicalAcademic', 'DegreeMaster', 'DegreeDoctor', 
         'Employment_Workplace_University', 'Employment_Workplace_Faculty', 
         'Employment_Type', 'Employment_Ratio'
        ]]
display(current_employees)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Ratio
0,18799121690,Kočan,Štefan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,None,Docent,100
1,16290288493,Stieranka,Jozef,prof.,Ing.,PhD.,Akadémia Policajného zboru,None,Profesor,100
2,21401303068,Wäldl,Ľuboš,doc.,JUDr.,PhD.,Akadémia Policajného zboru,None,Profesor,100
3,69828748660,Kostrec,Matej,None,JUDr.,PhD.,Akadémia Policajného zboru,None,Odborný asistent,100
4,91221451321,Krajníková,Magdaléna,None,JUDr.,PhD.,Akadémia Policajného zboru,None,Odborný asistent,100
...,...,...,...,...,...,...,...,...,...,...
13730,12110964830,Spurný,Joža,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,None,Docent,100
13731,13500390262,Harčárová,Marcela,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,None,Asistent,50
13732,17108478442,Juhásová,Jana,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,None,Asistent,50
13733,15917135755,Kubušová,Miriama,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,None,Asistent,50


#### - Momentálne máme údaje o viacerých zamestnaniach jedného zamestnanca uložené ako string alebo list v stĺpci Employment_Workplace_University
#### - Rozložme tieto dáta na viacero riadkov pre každé jedno zamestnanie:

In [12]:
df_workplaces = []
weird_data = []

for i, row in current_employees.iterrows():
    if type(row['Employment_Workplace_University']) == list:
        unis = row['Employment_Workplace_University']
        faculties = row['Employment_Workplace_Faculty']
        for u, university in enumerate(unis):
            new_row = row.copy()
            new_row['Employment_Workplace_University'] = university
            if faculties is not None:
                try:
                    faculty = faculties[u]
                    new_row['Employment_Workplace_Faculty'] = faculty 
                    new_row['Employment_Ratio'] = row['Employment_Ratio'][u]
                    new_row['Employment_Type'] = row['Employment_Type'][u]
                    
                    df_workplaces.append(new_row)
                except IndexError:
                    weird_data.append(row)
    else:
        new_row = row.copy()
        df_workplaces.append(new_row)

In [13]:
df_workplaces = pd.DataFrame(df_workplaces, columns=current_employees.columns)
display(df_workplaces)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Ratio
0,18799121690,Kočan,Štefan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,None,Docent,100
1,16290288493,Stieranka,Jozef,prof.,Ing.,PhD.,Akadémia Policajného zboru,None,Profesor,100
2,21401303068,Wäldl,Ľuboš,doc.,JUDr.,PhD.,Akadémia Policajného zboru,None,Profesor,100
3,69828748660,Kostrec,Matej,None,JUDr.,PhD.,Akadémia Policajného zboru,None,Odborný asistent,100
4,91221451321,Krajníková,Magdaléna,None,JUDr.,PhD.,Akadémia Policajného zboru,None,Odborný asistent,100
...,...,...,...,...,...,...,...,...,...,...
13730,12110964830,Spurný,Joža,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,None,Docent,100
13731,13500390262,Harčárová,Marcela,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,None,Asistent,50
13732,17108478442,Juhásová,Jana,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,None,Asistent,50
13733,15917135755,Kubušová,Miriama,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,None,Asistent,50


#### Problematické dáta vyriešime po jednom:

In [14]:
# for row in weird_data:
#     display(row)
weird_data_ids = [12477532568, 17628642100, 28653015119]

In [15]:
df[df['Employee_ID']=='12477532568']

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_EndDate,Employment_Ratio,Guarantee,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy
1987,12477532568,Durdík,Štefan,Dr. h. c,MUDr.,"PhD., MHA","[Univerzita Komenského v Bratislave, Univerzit...","[Lekárska fakulta, Fakulta zdravotníckych vied]","[Profesor, Profesor, Profesor]",[s titulom profesor a vysokoškolským vzdelaním...,"[2015-01-01, 2022-07-01, 2020-10-01]",2022-09-30,"[100, 5, 50]",True,"[chirurgia, chirurgia]","[Univerzita Komenského v Bratislave, Univerzit...","[Lekárska fakulta, Lekárska fakulta]","[1, 2]","[3, 3]"


In [16]:
weird_data = [["12477532568", 'Štefan', 'Durdík', 'Dr. h. c', 'MUDr.', 'PhD., MHA', 'Univerzita Komenského v Bratislave', 'Lekárska fakulta',
              'Profesor', 100], 
              ["12477532568", 'Štefan', 'Durdík', 'Dr. h. c', 'MUDr.', 'PhD., MHA', 'Univerzita sv. Cyrila a Metoda v Trnave',
              'Fakulta zdravotníckych vied', 'Profesor', 5], 
              ["12477532568", 'Štefan', 'Durdík', 'Dr. h. c', 'MUDr.', 'PhD., MHA', 'Vysoká škola zdravotníctva a sociálnej práce sv. Alžbety v Bratislave, n. o.',
              None, 'Profesor', 50], 
              ['17628642100', 'Peter', 'Fabo', None, 'RNDr.', 'PhD.', 'Trenčianska univerzita Alexandra Dubčeka v Trenčíne',
              'Fakulta špeciálnej techniky', 'Odborný asistent', 30],
              ['17628642100', 'Peter', 'Fabo', None, 'RNDr.', 'PhD.', 'Žilinská univerzita v Žiline',
              None, 'Výskumný pracovník - výskumník', 20],
              ['17628642100', 'Peter', 'Fabo', None, 'RNDr.', 'PhD.', 'Trenčianska univerzita Alexandra Dubčeka v Trenčíne',
              'Fakulta špeciálnej techniky', 'Výskumný pracovník - výskumník', 100],
              ['28653015119', 'Marianna', 'Berinšterová', None, 'Mgr.', 'PhD.', 'Prešovská univerzita v Prešove',
              'Fakulta humanitných a prírodných vied', 'Odborný asistent', 100], 
              ['28653015119', 'Marianna', 'Berinšterová', None, 'Mgr.', 'PhD.', 'Vysoká škola medzinárodného podnikania ISM Slovakia v Prešove',
              None, 'Zamestnanec vykonávajúci činnosti VŠ učiteľa mimo prac. pomeru', 5], 
              ['28653015119', 'Marianna', 'Berinšterová', None, 'Mgr.', 'PhD.', 'Prešovská univerzita v Prešove',
              'Fakulta športu', 'Zamestnanec vykonávajúci činnosti VŠ učiteľa mimo prac. pomeru', 15.55] 
             ]
small_dataframe =  pd.DataFrame(weird_data, columns=['Employee_ID', 'FamilyName', 'GivenName', 'DegreePedagogicalAcademic', 
                                                'DegreeMaster', 'DegreeDoctor', 'Employment_Workplace_University', 'Employment_Workplace_Faculty', 
                                                'Employment_Type', 'Employment_Ratio'])
display(small_dataframe)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Ratio
0,12477532568,Štefan,Durdík,Dr. h. c,MUDr.,"PhD., MHA",Univerzita Komenského v Bratislave,Lekárska fakulta,Profesor,100.00
1,12477532568,Štefan,Durdík,Dr. h. c,MUDr.,"PhD., MHA",Univerzita sv. Cyrila a Metoda v Trnave,Fakulta zdravotníckych vied,Profesor,5.00
2,12477532568,Štefan,Durdík,Dr. h. c,MUDr.,"PhD., MHA",Vysoká škola zdravotníctva a sociálnej práce s...,None,Profesor,50.00
3,17628642100,Peter,Fabo,None,RNDr.,PhD.,Trenčianska univerzita Alexandra Dubčeka v Tre...,Fakulta špeciálnej techniky,Odborný asistent,30.00
4,17628642100,Peter,Fabo,None,RNDr.,PhD.,Žilinská univerzita v Žiline,None,Výskumný pracovník - výskumník,20.00
5,17628642100,Peter,Fabo,None,RNDr.,PhD.,Trenčianska univerzita Alexandra Dubčeka v Tre...,Fakulta špeciálnej techniky,Výskumný pracovník - výskumník,100.00
6,28653015119,Marianna,Berinšterová,None,Mgr.,PhD.,Prešovská univerzita v Prešove,Fakulta humanitných a prírodných vied,Odborný asistent,100.00
7,28653015119,Marianna,Berinšterová,None,Mgr.,PhD.,Vysoká škola medzinárodného podnikania ISM Slo...,None,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,5.00
8,28653015119,Marianna,Berinšterová,None,Mgr.,PhD.,Prešovská univerzita v Prešove,Fakulta športu,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,15.55


In [17]:
df_workplaces_final = pd.concat([df_workplaces, small_dataframe], axis=0).reset_index(drop=True).drop(column="Employment_Workplace_Facult)
display(df_workplaces_final)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Ratio
0,18799121690,Kočan,Štefan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,None,Docent,100
1,16290288493,Stieranka,Jozef,prof.,Ing.,PhD.,Akadémia Policajného zboru,None,Profesor,100
2,21401303068,Wäldl,Ľuboš,doc.,JUDr.,PhD.,Akadémia Policajného zboru,None,Profesor,100
3,69828748660,Kostrec,Matej,None,JUDr.,PhD.,Akadémia Policajného zboru,None,Odborný asistent,100
4,91221451321,Krajníková,Magdaléna,None,JUDr.,PhD.,Akadémia Policajného zboru,None,Odborný asistent,100
...,...,...,...,...,...,...,...,...,...,...
14326,17628642100,Peter,Fabo,None,RNDr.,PhD.,Žilinská univerzita v Žiline,None,Výskumný pracovník - výskumník,20.0
14327,17628642100,Peter,Fabo,None,RNDr.,PhD.,Trenčianska univerzita Alexandra Dubčeka v Tre...,Fakulta špeciálnej techniky,Výskumný pracovník - výskumník,100.0
14328,28653015119,Marianna,Berinšterová,None,Mgr.,PhD.,Prešovská univerzita v Prešove,Fakulta humanitných a prírodných vied,Odborný asistent,100.0
14329,28653015119,Marianna,Berinšterová,None,Mgr.,PhD.,Vysoká škola medzinárodného podnikania ISM Slo...,None,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,5.0


In [19]:
df_workplaces_final.to_csv('../data/preprocessed/regzam_workplaces_current.csv')